# 自然数をジグザグに並べて円周率を求める



$1$ から $n$ までの自然数を並べて、前から順に $a_1, a_2, \cdots, a_n$ とおきます。$a_1, a_2, \cdots, a_n$ が

$$\begin{align}
a_1 < a_2 > a_3 < \cdots
\end{align}$$

を満たすとき、その並べ方をジグザグ順列といいます。$1$ から $n$ までの自然数のジグザグ順列の数を $e_n$ とおくと

$$\begin{align}
\lim_{n\to\infty} \frac{n e_{n-1}}{e_{n}} = \frac{\pi}{2}
\end{align}$$

が成り立つことが知られています。この式から円周率が計算できるか検証します。

まずは愚直に、自然数の全ての順列からジグザグ順列を抜き出して $e_n$ を数えます。

In [25]:
from itertools import permutations


def count_zugzag_perm(n: int) -> int: 

    def is_zigzag(perm: [int]) -> bool:
        prev = perm[0]
        for i_, a in enumerate(perm[1:]):
            i = i_ + 1
            
            if i % 2 == 1 and prev < a:
                # a_{2n} < a_{2n+1} なのでfalse
                return False
            elif i % 2 == 0 and prev > a:
                # a_{2n-1} > a_{2n} なのでfalse
                return False       

            prev = a
            
        return True

    return len(
        [
            perm
            for perm in permutations(range(1, n+1), n)
            if is_zigzag(perm)
        ]
    )

In [27]:
for n in range(1, 10):
    print(f"e_{n} = {count_zugzag_perm(n)}")

e_1 = 1
e_2 = 1
e_3 = 2
e_4 = 5
e_5 = 16
e_6 = 61
e_7 = 272
e_8 = 1385
e_9 = 7936


ただし、この求め方は効率が悪く、別の方法で求めるのが良いです。実は

$$2e_{n+1} = \sum_{k=0}^{n}\binom{n}{k} e_{k} e_{n-k}$$

という漸化式が成り立つことが知られています。この方法で計算してみましょう。

In [10]:
from functools import lru_cache
import math

@lru_cache(maxsize=1000)
def zigzag_num(n: int) -> int:
    if n < 0:
         raise ValueError(f"Input: n = {n}. Input value must be > 0.")

    if n == 0 or n == 1:
        return 1
    else:
        return sum([
            math.comb(n - 1, k) * zigzag_num(k) * zigzag_num(n -1 -k)
            for k in range(0, n)
        ]) // 2

In [30]:
print(f" \t 漸化式 \t カウント")
for n in range(1, 10):
    print(f"e_{n}\t {zigzag_num(n)} \t {count_zugzag_perm(n)}")

 	 漸化式 	 カウント
e_1	 1 	 1
e_2	 1 	 1
e_3	 2 	 2
e_4	 5 	 5
e_5	 16 	 16
e_6	 61 	 61
e_7	 272 	 272
e_8	 1385 	 1385
e_9	 7936 	 7936


$e_n$ の指数型母関数を

$$f(x) = \sum_{n=0}^{\infty} \frac{e_n}{n!} x^n$$

とおくと、

$$f(x) = \tan x + \frac{1}{\cos x}$$

が成り立つことが知られています。これにより $f(x)$ 収束半径は $\frac{\pi}{2}$ であることがわかり

$$\begin{align}
\lim_{n\to\infty} \frac{n e_{n-1}}{e_n} &= \frac{\pi}{2} \quad (ただし極限が存在すれば) \\
\liminf_{n\to\infty} \sqrt[n]{\frac{n!}{e_n}} &= \frac{\pi}{2}
\end{align}$$

となります。実際に計算すると、以下のようになります。

In [19]:
def dAlembert(n: int) -> float:
    return 2 * (n+1) * zigzag_num(n) / zigzag_num(n+1)

def cauchy_hadamard(n: int) -> float:
    return 2 * math.pow(
        math.factorial(n) / zigzag_num(n),
        1 / n
    )

In [32]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import print_pi as print_pi_, PI_50
DPS = 50

N = 10

print_pi = lambda x: print_pi_(x, PI_50, DPS, _format=f"n = {n}: "+"{pi} ({mdigit} 桁まで一致)")

print("d'Alembert")
for n in range(1, N +1):
    print_pi(dAlembert(n))

print("\nCauchyHadamard")
for n in range(1, N +1):
    print_pi(cauchy_hadamard(n))

print("\nCauchyHadamard")
for n in range(1, 500, 40):
    print_pi(cauchy_hadamard(n))

d'Alembert
n = 1: 4.0 (0 桁まで一致)
n = 2: 3.0 (0 桁まで一致)
n = 3: 3.2 (0 桁まで一致)
n = 4: 3.125 (1 桁まで一致)
n = 5: 3.1475409836065573 (2 桁まで一致)
n = 6: 3.139705882352941 (1 桁まで一致)
n = 7: 3.1422382671480142 (2 桁まで一致)
n = 8: 3.141381048387097 (3 桁まで一致)
n = 9: 3.141663862552206 (3 桁まで一致)
n = 10: 3.1415690575253254 (4 桁まで一致)

CauchyHadamard
n = 1: 2.0 (0 桁まで一致)
n = 2: 2.8284271247461903 (0 桁まで一致)
n = 3: 2.8844991406148166 (0 桁まで一致)
n = 4: 2.960331217969141 (0 桁まで一致)
n = 5: 2.9925557394776896 (0 桁まで一致)
n = 6: 3.0178462212865687 (0 桁まで一致)
n = 7: 3.0349607998067603 (0 桁まで一致)
n = 8: 3.048167578315654 (0 桁まで一致)
n = 9: 3.0583865177718805 (0 桁まで一致)
n = 10: 3.066613937134974 (0 桁まで一致)

CauchyHadamard
n = 1: 2.0 (0 桁まで一致)
n = 41: 3.1231373871598715 (1 桁まで一致)
n = 81: 3.1322375093429717 (1 桁まで一致)
n = 121: 3.135327032487838 (1 桁まで一致)
n = 161: 3.1368825409148418 (1 桁まで一致)
n = 201: 3.137819313604007 (1 桁まで一致)
n = 241: 3.138445280267887 (1 桁まで一致)
n = 281: 3.1388931123886015 (1 桁まで一致)
n = 321: 3.1392293772556084 (1 桁